In [43]:
from dataset import DataIterator, load_dataset
import numpy as np
from matplotlib import pyplot as plt
from skimage import color

data_path = "./extra_data/Alizarine/folds.json"
train, test = load_dataset(data_path, True)[0]

np.random.seed(10)

dataset_mask = DataIterator(
    train, 32, patch_per_image=32, inv_values=False).get_dataset()

dataset_test = DataIterator(
    test, 32, patch_per_image=64, inv_values=False).get_dataset()

images, masks = dataset_test[1], dataset_mask[0]


validation_data = np.asarray(
    [color.gray2rgb((im[..., 0]+1) / 2 * 255) for im in images], dtype=np.uint8)
# validation_data = np.transpose(validation_data, (0, 3, 1, 2))

masks.shape, images.shape, validation_data.min(), validation_data.max()

((640, 64, 64, 1), (640, 64, 64, 1), 18, 193)

In [95]:
from pathlib import Path
from keras import Model, Input, Sequential
from keras.models import load_model
import torch
from skimage import io
_ = torch.manual_seed(123)
from torchmetrics.image.fid import FrechetInceptionDistance

def remove_gaussian_from_model(model: Model) -> Model:
    lay_1 = model.get_layer('conv2d_7')
    lay_2 = model.get_layer('output')
    x = lay_1(model.layers[-4].output)
    x = lay_2(x)
    new_model = Model(inputs=model.input, outputs=x)
    return new_model

imgs_dist1 = torch.from_numpy(validation_data)

pd1 = f'R:/pd1/'
pd2 = f'R:/pd2/'
pdt = f'R:/pdt/'

for k in range(len(validation_data)):
    io.imsave(f'{pdt}{k}.png', validation_data[k])

score1 = []
score2 = []
for i in range(21, 101, 10):
    path1 = f'F:/Deep Learning/GAN-CNN-CE/generator/models/20220405-2359/model_{i}.h5'
    path2 = f'R:/fit/20230318-025846/generators/gan_{i}.hdf5'
    generator1: Model = remove_gaussian_from_model(load_model(path1))
    generator2: Model = load_model(path2)

    
    new_mask1 = generator1.predict_on_batch([1 - masks, np.random.normal(size=masks.shape)])
    imgs1 = np.asarray([color.gray2rgb((im[..., 0]+1) / 2 * 255) for im in new_mask1], dtype=np.uint8)
    for k in range(len(imgs1)):
        fp = f'{pd1}{i}/'
        Path(fp).mkdir(exist_ok=True, parents=True)
        io.imsave(f'{fp}{k}.png', imgs1[k])

    new_mask2 = generator2.predict_on_batch(masks)
    imgs2 = np.asarray([color.gray2rgb((im[..., 0]+1) / 2 * 255) for im in new_mask2], dtype=np.uint8)
    for k in range(len(imgs2)):
        fp = f'{pd2}{i}/'
        Path(fp).mkdir(exist_ok=True, parents=True)
        io.imsave(f'{fp}{k}.png', imgs2[k])

    

0

: 